## installation and set up

In [ ]:
!pip install openai

from google.colab import drive
drive.mount('/content/drive')

In [28]:
# setup
import time
import json
from openai import OpenAI
import textwrap

class Assistant:
    def __init__(self, *, api_key, output_folder="/content/drive/MyDrive", assistant_id):
        self.OPENAI_API_KEY = api_key
        self.output_folder = output_folder
        self.id = assistant_id
        self.client = OpenAI(api_key=self.OPENAI_API_KEY)
        # if self.id == "new" then create a new one?

    def text_to_text(self, prompt):
      thread_id = self.new_thread()
      run_id = self.add_message_and_run(thread_id, prompt)
      run_info = self.wait_for_completion(thread_id, run_id)
      message_text = self.get_thread_message(thread_id)
      return message_text

    # New thread method to create a thread and send an initial message
    def new_thread(self):
        thread = self.client.beta.threads.create()
        return thread.id

    def add_message_and_run(self, thread_id, prompt):
        message = self.client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=prompt
        )
        run = self.client.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=self.id
        )
        return run.id


    # Method for checking the status until the run is completed
    def wait_for_completion(self, thread_id, run_id):
        while True:
            run_info = self.client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            print(f"Current run status: {run_info.status}")
            if run_info.status == "completed":
                print("Run is complete.")
                break
            else:
                # Inform about the ongoing wait and current status
                print(f"not done yet, status is {run_info.status}")
                time.sleep(5)  # Adjust sleep time as needed

        return run_info  # Return run_info for possible further processing

    def get_thread_message(self, thread_id):
        messages = self.client.beta.threads.messages.list(thread_id=thread_id)
        message_info = self.client.beta.threads.messages.retrieve(
                thread_id=thread_id,
                message_id=messages[0].id
            )
        return message_info

    def p_dict(self, obj):
      print(json.dumps(obj.dict(), indent=4))

    def p_wrapped(self, text):
      print(textwrap(text, width=50))



In [29]:
from google.colab import userdata
from google.colab import drive

neanderthal_id = userdata.get('neanderthal_assistant_id')
proto_indo_european_id = userdata.get('proto_indo_european_assistant_id')
rising_star_id = userdata.get('rising_star_assistant_id')
sanxingdui_id = userdata.get('sanxingdui_assistant_id')

group1_assistants = {
    "neanderthal": neanderthal_id,
    "proto_indo_european": proto_indo_european_id,
    "rising_star": rising_star_id,
    "sanxingdui": sanxingdui_id
}

Neanderthal = Assistant(assistant_id=neanderthal_id, api_key=userdata.get("GROUP1_OPENAI_API_KEY"))

In [ ]:
result = Neanderthal.text_to_text("tell me a story of an eclipse")


## creating and updating assistants

In [ ]:
from openai import OpenAI
import json

assistant = client.beta.assistants.create(
    name="demo-bot-001",
    instructions="you are a bot who possesses knowledge of folklore and has the ability to retrieve information from texts using the OpenAI Assistant retrieval tool. Whenever possible try to retrieve information from these texts in order to augment your responses.",
    tools=[{"type": "retrieval"}],
    model="gpt-4-turbo-preview"
)

print(assistant)


### upload relevant documents

In [ ]:
import os

# Path to your assistant documents
assistant_docs_path = "/content/drive/MyDrive"

# Initialize an empty list to hold the file IDs
uploaded_file_ids = []

# Iterate over each file in the directory
for filename in os.listdir(assistant_docs_path):
    print(filename)
    file_path = os.path.join(assistant_docs_path, filename)
    print(file_path)

    # Check if it's a file (and not a directory)
    if os.path.isfile(file_path):
        print(f"Uploading {filename}...")
        # Upload the file and store the response
        response = client.files.create(
            file=open(file_path, "rb"),
            purpose="assistants"
        )

        print(response.dict())
        uploaded_file_ids.append(response.id)
        print(f"Uploaded {filename} and received id: {response.id}")

# Now, uploaded_file_ids contains all the uploaded file IDs
print("Uploaded file IDs:", uploaded_file_ids)

# Use uploaded_file_ids for training the assistant as needed

In [ ]:
file_ids = uploaded_file_ids # from previous step
new_assistant_name = "demo_bot_now_with_retrieval"

my_updated_assistant = client.beta.assistants.update(
  assistant.id,
  name=new_assistant_name,
  tools=[{"type": "retrieval"}],
  model="gpt-4-turbo-preview",
  file_ids=file_ids,
)



In [ ]:
print(json.dumps(my_updated_assistant.dict(), indent=4))

In [ ]:
function_schema={
  "type": "function",
  "function": {
      "name": "generate_narrative_structure",
      "description": "Generates a narrative structure based on selected narratemes and stages.",
      "parameters": {
          "type": "object",
          "properties": {
              "color_theme": {
                  "type": "string",
                  "enum": ["red", "green", "blue", "yellow", "white", "black"],
                  "description": "The color theme, influencing the narrative's mood and elements, referecing Goethe's color theory.",
              },
              "narratemes": {
                  "type": "array",
                  "items": {
                      "type": "string",
                      "enum": [
                          "absentation", "interdiction", "violation", "reconnaissance",
                          "delivery", "trickery", "complicity", "villainy", "mediation",
                          "beginning counteraction", "departure", "first function of the donor",
                          "hero's reaction", "receipt of a magical agent", "guidance", "struggle",
                          "branding", "victory", "liquidation", "return", "pursuit", "rescue",
                          "unrecognized arrival", "unfounded claims", "difficult task",
                          "solution", "recognition", "exposure", "transfiguration",
                          "punishment", "wedding"
                      ],
                      "description": "Select Propp's narratemes to include in the narrative structure.",
                  },
                  "minItems": 1,
                  "uniqueItems": True
              },
              "stages": {
                  "type": "array",
                  "items": {
                      "type": "string",
                      "enum": [
                          "call to adventure", "refusal of the call", "supernatural aid",
                          "crossing the first threshold", "belly of the whale", "the road of trials",
                          "meeting with the goddess", "woman as temptress", "atonement with the father",
                          "apotheosis", "the ultimate boon", "refusal of the return",
                          "the magic flight", "rescue from without", "the crossing of the return threshold",
                          "master of two worlds", "freedom to live"
                      ],
                      "description": "Select Campbell's stages to weave into the narrative structure.",
                  },
                  "minItems": 1,
                  "uniqueItems": True
              }
          },
          "required": ["color_theme", "narratemes", "stages"]
      }
  }

In [ ]:
import os

# Path to your assistant documents
assistant_docs_path = "/content/drive/MyDrive"

# Initialize an empty list to hold the file IDs
uploaded_file_ids = []

# Iterate over each file in the directory
for filename in os.listdir(assistant_docs_path):
    print(filename)
    file_path = os.path.join(assistant_docs_path, filename)
    print(file_path)

    # Check if it's a file (and not a directory)
    if os.path.isfile(file_path):
        print(f"Uploading {filename}...")
        # Upload the file and store the response
        response = client.files.create(
            file=open(file_path, "rb"),
            purpose="assistants"
        )

        print(response.dict())
        uploaded_file_ids.append(response.id)
        print(f"Uploaded {filename} and received id: {response.id}")

# Now, uploaded_file_ids contains all the uploaded file IDs
print("Uploaded file IDs:", uploaded_file_ids)

# Use uploaded_file_ids for training the assistant as needed


In [ ]:
file_ids = uploaded_file_ids # from previous step
new_assistant_name = "demo_bot_now_with_retrieval"

my_updated_assistant = client.beta.assistants.update(
  assistant.id,
  name=new_assistant_name,
  tools=[{"type": "retrieval"}],
  model="gpt-4-turbo-preview",
  file_ids=file_ids,
)

In [ ]:
function_schema={
  "type": "function",
  "function": {
      "name": "generate_narrative_structure",
      "description": "Generates a narrative structure based on selected narratemes and stages.",
      "parameters": {
          "type": "object",
          "properties": {
              "color_theme": {
                  "type": "string",
                  "enum": ["red", "green", "blue", "yellow", "white", "black"],
                  "description": "The color theme, influencing the narrative's mood and elements, referecing Goethe's color theory.",
              },
              "narratemes": {
                  "type": "array",
                  "items": {
                      "type": "string",
                      "enum": [
                          "absentation", "interdiction", "violation", "reconnaissance",
                          "delivery", "trickery", "complicity", "villainy", "mediation",
                          "beginning counteraction", "departure", "first function of the donor",
                          "hero's reaction", "receipt of a magical agent", "guidance", "struggle",
                          "branding", "victory", "liquidation", "return", "pursuit", "rescue",
                          "unrecognized arrival", "unfounded claims", "difficult task",
                          "solution", "recognition", "exposure", "transfiguration",
                          "punishment", "wedding"
                      ],
                      "description": "Select Propp's narratemes to include in the narrative structure.",
                  },
                  "minItems": 1,
                  "uniqueItems": True
              },
              "stages": {
                  "type": "array",
                  "items": {
                      "type": "string",
                      "enum": [
                          "call to adventure", "refusal of the call", "supernatural aid",
                          "crossing the first threshold", "belly of the whale", "the road of trials",
                          "meeting with the goddess", "woman as temptress", "atonement with the father",
                          "apotheosis", "the ultimate boon", "refusal of the return",
                          "the magic flight", "rescue from without", "the crossing of the return threshold",
                          "master of two worlds", "freedom to live"
                      ],
                      "description": "Select Campbell's stages to weave into the narrative structure.",
                  },
                  "minItems": 1,
                  "uniqueItems": True
              }
          },
          "required": ["color_theme", "narratemes", "stages"]
      }
  }